In [1]:
import tiktoken
import torch
torch.manual_seed(123)
tokenizer = tiktoken.get_encoding(encoding_name="gpt2")

# Input Text/Encoding

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [3]:
raw_text[:20]

'I HAD always thought'

In [4]:
tokenizer.decode([tokenizer.eot_token])

'<|endoftext|>'

In [5]:
text = raw_text[:20] + '<|endoftext|>'
encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
inputs = torch.tensor(encoded)
text, inputs

('I HAD always thought<|endoftext|>',
 tensor([   40,   367,  2885,  1464,  1807, 50256]))

# Token Embeddings

In [6]:
vocab_size = tokenizer.n_vocab
output_dim = 3
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [7]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings, token_embeddings.shape

(tensor([[ 1.3966e+00, -9.9491e-01, -1.5822e-03],
         [-1.1659e+00,  1.3834e-01, -9.8013e-01],
         [ 1.0122e+00, -6.5152e-01,  8.6051e-02],
         [ 2.5264e-01,  2.1043e+00, -6.4511e-01],
         [-3.1843e-01, -1.2264e-01, -3.9307e-02],
         [-3.1300e-01,  7.5582e-01, -1.2656e+00]], grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 3]))

# Position Embeddings

In [8]:
max_length = len(inputs)
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [9]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
pos_embeddings, pos_embeddings.shape

(tensor([[ 0.1583,  1.0919,  0.8055],
         [ 1.1262, -0.8857, -1.6315],
         [-0.2570,  0.7127, -0.3622],
         [ 0.9609, -1.3697,  0.1381],
         [-1.2365,  1.9319,  0.4730],
         [ 0.7365,  0.1316,  0.2379]], grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 3]))

# Add them up!

In [10]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings, input_embeddings.shape

(tensor([[ 1.5549,  0.0969,  0.8039],
         [-0.0397, -0.7474, -2.6117],
         [ 0.7552,  0.0611, -0.2762],
         [ 1.2135,  0.7346, -0.5071],
         [-1.5549,  1.8092,  0.4337],
         [ 0.4235,  0.8874, -1.0277]], grad_fn=<AddBackward0>),
 torch.Size([6, 3]))

# Simple self-attention (no trainable weights)

In [11]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.10], # one
        [0.05, 0.80, 0.55], # step
    ]
)

In [12]:
query = inputs[1] # Second input token will be our query
query

tensor([0.5500, 0.8700, 0.6600])

In [13]:
inputs.shape

torch.Size([6, 3])

In [14]:
attn_scores_2 = torch.empty(inputs.shape[0])
attn_scores_2

tensor([0., 0., 0., 0., 0., 0.])

In [15]:
query

tensor([0.5500, 0.8700, 0.6600])

In [16]:
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query)
attn_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [17]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=-1)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [18]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
query, query.shape, context_vec_2
for i, x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i] * x_i
  print(f"{attn_weights_2[i] * x_i} = {attn_weights_2[i]:.2f} * {x_i}")
print(context_vec_2)

tensor([0.0596, 0.0208, 0.1233]) = 0.14 * tensor([0.4300, 0.1500, 0.8900])
tensor([0.1308, 0.2070, 0.1570]) = 0.24 * tensor([0.5500, 0.8700, 0.6600])
tensor([0.1330, 0.1983, 0.1493]) = 0.23 * tensor([0.5700, 0.8500, 0.6400])
tensor([0.0273, 0.0719, 0.0409]) = 0.12 * tensor([0.2200, 0.5800, 0.3300])
tensor([0.0833, 0.0270, 0.0108]) = 0.11 * tensor([0.7700, 0.2500, 0.1000])
tensor([0.0079, 0.1265, 0.0870]) = 0.16 * tensor([0.0500, 0.8000, 0.5500])
tensor([0.4419, 0.6515, 0.5683])


In [19]:
attn_scores = inputs @ inputs.T
mask = attn_scores == attn_scores.max(dim=0, keepdim=True).values
print(mask)

attn_scores = attn_scores * mask
print(attn_scores)

tensor([[ True, False, False, False, False, False],
        [False,  True,  True,  True, False,  True],
        [False, False, False, False,  True, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])
tensor([[0.9995, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.4950, 1.4754, 0.8434, 0.0000, 1.0865],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7154, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])


In [20]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [21]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [22]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [23]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [24]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


In [25]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
x_2, d_in, d_out

(tensor([0.5500, 0.8700, 0.6600]), 3, 2)

In [26]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [27]:
query_2 = x_2 @ W_query
key_2   = x_2 @ W_key
value_2 = x_2 @ W_value
query_2, key_2, value_2

(tensor([0.4306, 1.4551]), tensor([0.4433, 1.1419]), tensor([0.3951, 1.0037]))

In [28]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)
print(keys, query_2)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])
tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]]) tensor([0.4306, 1.4551])


In [29]:
keys_2 = keys[1]
attn_scores_22 = query_2.dot(keys_2)
attn_scores_22, keys_2

(tensor(1.8524), tensor([0.4433, 1.1419]))

In [30]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [31]:
print(query_2)
print(keys.T)

tensor([0.4306, 1.4551])
tensor([[0.3669, 0.4433, 0.4361, 0.2408, 0.1827, 0.3275],
        [0.7646, 1.1419, 1.1156, 0.6706, 0.3292, 0.9642]])


In [32]:
print(query_2, keys.T[:, 0], query_2 @ keys.T[:,0])

tensor([0.4306, 1.4551]) tensor([0.3669, 0.7646]) tensor(1.2705)


In [33]:
d_k = keys.shape[-1]
d_k

2

In [34]:
attn_weights_2  = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [35]:
values

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])

In [36]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([0.3061, 0.8210])

# Next: 3.4.2 (page 92)

# Self-attention

In [37]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()

        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys    = x @ self.W_key
        queries = x @ self.W_query

        attn_scores  = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        values      = x @ self.W_value
        context_vec = attn_weights @ values

        return context_vec

    def assign_params(self, q, k, v):
        self.W_query = nn.Parameter(q)
        self.W_key   = nn.Parameter(k)
        self.W_value = nn.Parameter(v)

In [38]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [39]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys    = self.W_key(x)
        queries = self.W_query(x)

        attn_scores  = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        values      = self.W_value(x)
        context_vec = attn_weights @ values

        return context_vec

    def weights(self):
        return self.W_query.weights

In [40]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [41]:
q, k, v = list(sa_v2.parameters())

In [42]:
q, k, v

(Parameter containing:
 tensor([[ 0.3161,  0.4568,  0.5118],
         [-0.1683, -0.3379, -0.0918]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.4058, -0.4704,  0.2368],
         [ 0.2134, -0.2601, -0.5105]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2526, -0.1415, -0.1962],
         [ 0.5191, -0.0852, -0.2043]], requires_grad=True))

In [43]:
sa_v1.assign_params(q.T, k.T, v.T)

In [44]:
sa_v1(inputs), sa_v2(inputs)

(tensor([[-0.0739,  0.0713],
         [-0.0748,  0.0703],
         [-0.0749,  0.0702],
         [-0.0760,  0.0685],
         [-0.0763,  0.0679],
         [-0.0754,  0.0693]], grad_fn=<MmBackward0>),
 tensor([[-0.0739,  0.0713],
         [-0.0748,  0.0703],
         [-0.0749,  0.0702],
         [-0.0760,  0.0685],
         [-0.0763,  0.0679],
         [-0.0754,  0.0693]], grad_fn=<MmBackward0>))

In [45]:
keys = sa_v2.W_key(inputs)
queries = sa_v2.W_query(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/keys.shape[-1], dim=-1)
print(attn_scores, attn_weights, sep="\n")

tensor([[ 0.2899,  0.0716,  0.0760, -0.0138,  0.1344, -0.0511],
        [ 0.4656,  0.1723,  0.1751,  0.0259,  0.1771,  0.0085],
        [ 0.4594,  0.1703,  0.1731,  0.0259,  0.1745,  0.0090],
        [ 0.2642,  0.1024,  0.1036,  0.0186,  0.0973,  0.0122],
        [ 0.2183,  0.0874,  0.0882,  0.0177,  0.0786,  0.0144],
        [ 0.3408,  0.1270,  0.1290,  0.0198,  0.1290,  0.0078]],
       grad_fn=<MmBackward0>)
tensor([[0.1844, 0.1653, 0.1657, 0.1584, 0.1706, 0.1555],
        [0.1926, 0.1663, 0.1666, 0.1546, 0.1667, 0.1532],
        [0.1922, 0.1663, 0.1666, 0.1548, 0.1667, 0.1534],
        [0.1808, 0.1667, 0.1668, 0.1599, 0.1663, 0.1594],
        [0.1781, 0.1668, 0.1669, 0.1611, 0.1661, 0.1609],
        [0.1853, 0.1665, 0.1667, 0.1578, 0.1667, 0.1569]],
       grad_fn=<SoftmaxBackward0>)


In [46]:
sum([0.1844, 0.1653, 0.1657, 0.1584, 0.1706, 0.1555])

0.9999

In [47]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [48]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1844, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1926, 0.1663, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1922, 0.1663, 0.1666, 0.0000, 0.0000, 0.0000],
        [0.1808, 0.1667, 0.1668, 0.1599, 0.0000, 0.0000],
        [0.1781, 0.1668, 0.1669, 0.1611, 0.1661, 0.0000],
        [0.1853, 0.1665, 0.1667, 0.1578, 0.1667, 0.1569]],
       grad_fn=<MulBackward0>)


In [49]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5366, 0.4634, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3660, 0.3168, 0.3172, 0.0000, 0.0000, 0.0000],
        [0.2681, 0.2473, 0.2474, 0.2371, 0.0000, 0.0000],
        [0.2123, 0.1988, 0.1989, 0.1920, 0.1980, 0.0000],
        [0.1853, 0.1665, 0.1667, 0.1578, 0.1667, 0.1569]],
       grad_fn=<DivBackward0>)


In [50]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
mask.bool(), masked

(tensor([[False,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False, False,  True],
         [False, False, False, False, False, False]]),
 tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
         [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
         [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
         [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
         [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
        grad_fn=<MaskedFillBackward0>))

In [51]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [52]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [53]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


In [54]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [55]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys    = self.W_key(x)
        queries = self.W_query(x)
        values  = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [56]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [57]:
mask.bool()

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])

In [58]:
num_tokens = 6
mask.bool()[:num_tokens, :num_tokens]

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])

In [59]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)
print(context_vecs)

context_vecs.shape: torch.Size([2, 6, 2])
tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)


# Next: 3.6

In [60]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                CausalAttention(d_in, d_out, context_length, dropout, qkv_bias=qkv_bias)
                for _ in range(num_heads)
            ]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [61]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


# Next: 3.6.2
Improving the implementation by processing the heads in parallel, by computing the outputs for all the attention heads simultaneously via matrix multiplication.

In [71]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query  = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key    = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value  = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout  = nn.Dropout(dropout)

        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys: torch.Tensor    = self.W_key(x)
        queries: torch.Tensor = self.W_query(x)
        values: torch.Tensor  = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # b, num_tokens, num_heads, head_dim -> b, num_heads, num_tokens, head_dim
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # 0, 1,         2,          3
        # b, num_heads, num_tokens, head_dim
        attn_scores = queries @ keys.transpose(2, 3)

        # After matul:
        # b, num_heads, num_tokens, num_tokens

        # Apply causal mask
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # type: ignore
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # Softmax, logits -> prob
        attn_weights: torch.Tensor = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)

        # Apply dropout
        attn_weights: torch.Tensor = self.dropout(attn_weights)

        # b, num_heads, num_tokens, num_tokens @ b, num_heads, num_tokens, head_dim ->
        # b, num_heads, num_tokens, head_dim
        context_vec = attn_weights @ values
        # print("context_vec.shape:", context_vec.shape)
        # print("context_vec:", context_vec)

        # 0, 1,         2,          3
        # b, num_heads, num_tokens, head_dim -> b, num_tokens, num_heads, head_dim
        context_vec = context_vec.transpose(1, 2)

        # b, num_tokens, num_heads, head_dim -> b, num_tokens, d_out
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        context_vec: torch.Tensor = self.out_proj(context_vec)

        return context_vec

In [ ]:
a = torch.tensor(
    [
        [
            [
                [0.2745, 0.6584, 0.2775, 0.8573],
                [0.8993, 0.0390, 0.9268, 0.7388],
                [0.7179, 0.7058, 0.9156, 0.4340],
            ],
            [
                [0.0772, 0.3565, 0.1479, 0.5331],
                [0.4066, 0.2318, 0.4545, 0.9737],
                [0.4606, 0.5159, 0.4220, 0.5786],
            ],
        ]
    ]
)
print("a.shape:", a.shape)
print(a @ a.transpose(2, 3))

a.shape: torch.Size([1, 2, 3, 4])
tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


tensor(1.1631)

In [ ]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [72]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [76]:
batch_size, context_length, d_in = batch.shape

# GPT-2 params
d_out = 768
context_length = 1024
num_heads = 12

gpt2_mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads)
context_vecs = gpt2_mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.0609,  0.1996, -0.1692,  ..., -0.0019, -0.0838,  0.2607],
         [ 0.1341,  0.2619, -0.0928,  ...,  0.0266, -0.0920,  0.1994],
         [ 0.1586,  0.2787, -0.0634,  ...,  0.0383, -0.0952,  0.1743],
         [ 0.1421,  0.2541, -0.0574,  ...,  0.0361, -0.0754,  0.1557],
         [ 0.1510,  0.2136,  0.0057,  ...,  0.0623, -0.0952,  0.0924],
         [ 0.1369,  0.2248, -0.0252,  ...,  0.0460, -0.0754,  0.1197]],

        [[ 0.0609,  0.1996, -0.1692,  ..., -0.0019, -0.0838,  0.2607],
         [ 0.1341,  0.2619, -0.0928,  ...,  0.0266, -0.0920,  0.1994],
         [ 0.1586,  0.2787, -0.0634,  ...,  0.0383, -0.0952,  0.1743],
         [ 0.1421,  0.2541, -0.0574,  ...,  0.0361, -0.0754,  0.1557],
         [ 0.1510,  0.2136,  0.0057,  ...,  0.0623, -0.0952,  0.0924],
         [ 0.1369,  0.2248, -0.0252,  ...,  0.0460, -0.0754,  0.1197]]],
       grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 768])
